In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

### find all the combinations of sources and destinations

In [11]:
ind = "network_weather_2-2016.8.*"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}
response = es.search(index=ind, body=my_query, request_timeout=1200)
#print(response['aggregations']['type_counts']['buckets'])

In [12]:
# for cleaning specific links
limitTo=['192.41.230.60','192.41.236.32','192.41.230.59','192.41.236.31','128.142.223.247','128.142.223.246']

In [17]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    allPairs=0
    print(t['key'])
    if not t['key']=='traceroute': continue
    for s in t['src_counts']['buckets']:
        bulk_body = []
        dcounter=0
        for d in s['dest_counts']['buckets']:
            allPairs += 1
            src=s['key']
            dest=d['key']
            #if src not in limitTo and dest not in limitTo: continue
            #print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
            "size":100000,
#             "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
            "fields": ["timestamp"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                                    {"term": {"_type": {"value": t['key'] } } },
                                    {"term": {"src":   {"value": src } } },
                                    {"term": {"dest":  {"value": dest } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": [{ "timestamp": { "order": "asc" }}]
            }
            
            toDelete={}
#             oldres=0
            oldtime=""
            counter=0
            scroll = scan(es, query=myscroll, index=ind, size=5000, scroll='240s', request_timeout=120)
            for res in scroll:
#                 if counter==0: print(res)
                if res['fields']['timestamp'][0]==oldtime:# and res['fields']['src'][0]==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss'][0]
                oldtime=res['fields']['timestamp'][0]
                counter+=1
#                if counter>200:
#                    break
#                 if counter%100==0: 
                
            #print ("records:", counter, '\tto delete:',len(toDelete))
            for x in toDelete:
                if dcounter>100000:
                    dcounter=0
                    es.bulk('\n'.join(bulk_body), request_timeout=1200)
                    bulk_body=[]
                    print('deleted 100k')
                dcounter+=1
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
        if len(toDelete):
            print("final delete", dcounter)
            es.bulk('\n'.join(bulk_body), request_timeout=1200)

    print(t['key'],'Pairs:',allPairs)
print ('DONE.')

packet_loss_rate
latency
traceroute
final delete 410
final delete 271
final delete 807
final delete 727
final delete 658
final delete 220
final delete 450
final delete 356
final delete 65
final delete 170
final delete 137
final delete 154
final delete 55
final delete 69
final delete 67
final delete 76
final delete 69
final delete 61
final delete 77
final delete 82
final delete 79
final delete 74
final delete 80
final delete 79
final delete 72
final delete 84
final delete 75
final delete 77
final delete 79
final delete 67
traceroute Pairs: 7456
throughput
DONE.
